In [1]:
import csv
import numpy as np
import tensorflow as tf


pixel_depth = 255.0

def load_csv(filename,length,force=False):
    num = 0
    data = np.ndarray(shape=(length,48,48),dtype=np.float32)
    labels = np.ndarray(shape=(length,),dtype=np.float32)
    with open(filename) as csvfile:
        fer2013 = csv.reader(csvfile)
        for row in fer2013:
            if force == False:
                force = True
            else:
                d = row[1].split(" ")
                l = row[0]
                d = np.array(d,dtype=float).reshape((48,48))
                l = np.array(l,dtype=float)
                data[num,:,:] = d
                labels[num,]= l
                num = num + 1
                
    #normalized image dataset
    data = data[0:num,:,:]/255.0
    labels = labels[0:num,]
    print('Full dataset tensor:', data.shape)
    print('Label of dataset: ',labels.shape)
    print('Mean:', np.mean(data))
    print('Standard deviation:', np.std(data))
    return data,labels    



In [2]:
train_dataset,train_labels = load_csv("trainSample_v01.csv",10274)

Full dataset tensor: (10274, 48, 48)
Label of dataset:  (10274,)
Mean: 0.502654
Standard deviation: 0.252731


In [5]:
test_dataset,test_labels = load_csv("testSample_v01.csv",5918)



Full dataset tensor: (5918, 48, 48)
Label of dataset:  (5918,)
Mean: 0.502019
Standard deviation: 0.255782


In [6]:
valid_dataset,valid_labels = load_csv("validSample_v01.csv",5504)

Full dataset tensor: (5504, 48, 48)
Label of dataset:  (5504,)
Mean: 0.505731
Standard deviation: 0.253815


In [7]:
image_size = 48
num_labels = 7
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (10274, 48, 48, 1) (10274, 7)
Validation set (5504, 48, 48, 1) (5504, 7)
Test set (5918, 48, 48, 1) (5918, 7)


In [7]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [ ]:
#import tensorflow as tf

In [27]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
          tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [28]:
num_steps = 1000
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
         [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                  valid_prediction.eval(), valid_labels))
    x = test_prediction.eval()
    print('Test accuracy: %.1f%%' % accuracy(x, test_labels))

Initialized
Minibatch loss at step 0: 3.461625
Minibatch accuracy: 18.8%
Validation accuracy: 16.9%
Minibatch loss at step 50: 1.866313
Minibatch accuracy: 31.2%
Validation accuracy: 23.7%
Minibatch loss at step 100: 1.823701
Minibatch accuracy: 31.2%
Validation accuracy: 16.8%
Minibatch loss at step 150: 1.956679
Minibatch accuracy: 37.5%
Validation accuracy: 24.2%
Minibatch loss at step 200: 1.776879
Minibatch accuracy: 43.8%
Validation accuracy: 23.8%
Minibatch loss at step 250: 1.717361
Minibatch accuracy: 31.2%
Validation accuracy: 24.0%
Minibatch loss at step 300: 1.886068
Minibatch accuracy: 18.8%
Validation accuracy: 24.6%
Minibatch loss at step 350: 1.898278
Minibatch accuracy: 25.0%
Validation accuracy: 24.8%
Minibatch loss at step 400: 1.845861
Minibatch accuracy: 18.8%
Validation accuracy: 24.3%
Minibatch loss at step 450: 1.708178
Minibatch accuracy: 37.5%
Validation accuracy: 24.5%
Minibatch loss at step 500: 1.891434
Minibatch accuracy: 18.8%
Validation accuracy: 24.9%
M

In [29]:
type(x)

numpy.ndarray

In [32]:
x[1]

array([ 0.10190369,  0.04557588,  0.11921816,  0.23059921,  0.21533927,
        0.09604168,  0.19132213], dtype=float32)

In [8]:
#a = np.argmax(x,1)
b = np.argmax(train_labels,1)

In [3]:
import pandas as pd

In [10]:
b

array([2, 3, 6, ..., 2, 3, 5])

In [12]:
a_pd = pd.DataFrame({"a":a})

In [78]:
a_pd.columns

Index(['a'], dtype='object')

In [80]:
_0=np.sum(a_pd['a']==0)
_1=np.sum(a_pd['a']==1)
_2=np.sum(a_pd['a']==2)
_3=np.sum(a_pd['a']==3)
_4=np.sum(a_pd['a']==4)
_5=np.sum(a_pd['a']==5)
_6=np.sum(a_pd['a']==6)

In [110]:
print(_0,_1,_2,_3,_4,_5,_6)

215 573 82 3041 846 625 536


In [92]:
215+573+82+3041+846+625+536

5918

In [13]:
a_pd['b'] = b

In [14]:
b_0=np.sum(a_pd['b']==0)
b_1=np.sum(a_pd['b']==1)
b_2=np.sum(a_pd['b']==2)
b_3=np.sum(a_pd['b']==3)
b_4=np.sum(a_pd['b']==4)
b_5=np.sum(a_pd['b']==5)
b_6=np.sum(a_pd['b']==6)

In [15]:
print(b_0,b_1,b_2,b_3,b_4,b_5,b_6)

1280 1204 1336 2331 1632 898 1593


In [84]:
ind_0a =(a_pd['a']==0)
ind_1a =(a_pd['a']==1)
ind_2a =(a_pd['a']==2)
ind_3a =(a_pd['a']==3)
ind_4a =(a_pd['a']==4)
ind_5a =(a_pd['a']==5)
ind_6a =(a_pd['a']==6)

In [85]:
ind_0b =(a_pd['b']==0)
ind_1b =(a_pd['b']==1)
ind_2b =(a_pd['b']==2)
ind_3b =(a_pd['b']==3)
ind_4b =(a_pd['b']==4)
ind_5b =(a_pd['b']==5)
ind_6b =(a_pd['b']==6)

In [87]:
acc_0 = np.sum(ind_0a==ind_0b)
acc_1 = np.sum(ind_1a==ind_1b)
acc_2 = np.sum(ind_2a==ind_2b)
acc_3 = np.sum(ind_3a==ind_3b)
acc_4 = np.sum(ind_4a==ind_4b)
acc_5 = np.sum(ind_5a==ind_5b)
acc_6 = np.sum(ind_6a==ind_6b)

In [88]:
#print(acc_0,acc_1,acc_2,acc_3,acc_4,acc_5,acc_6)

4948 5317 5066 3382 4545 5102 4644


In [97]:
len(ind_0a)

5918

In [104]:
def accu(a,b):
    sum = 0
    for i in range(len(a)):
        if a[i]:
            if a[i]==b[i]:
                sum+=1
    return sum

In [105]:
acc_0 = accu(ind_0a,ind_1b)
acc_1 = accu(ind_1a,ind_1b)
acc_2 = accu(ind_2a,ind_2b)
acc_3 = accu(ind_3a,ind_3b)
acc_4 = accu(ind_4a,ind_4b)
acc_5 = accu(ind_5a,ind_5b)
acc_6 = accu(ind_6a,ind_6b)

In [112]:
print("categories          (0=An, 1=Dis, 2=Fe, 3=Ha, 4=Sa, 5=Sup, 6=Neutral).")
print(" actual labels count  ",b_0,b_1,b_2,b_3,b_4,b_5,b_6)
print("predicted labels count",_0,_1,_2,_3,_4,_5,_6)
print("accuracy              ",acc_0,acc_1,acc_2,acc_3,acc_4,acc_5,acc_6)


categories          (0=An, 1=Dis, 2=Fe, 3=Ha, 4=Sa, 5=Sup, 6=Neutral).
 actual labels count   833 86 824 1541 1037 563 1034
predicted labels count 215 573 82 3041 846 625 536
accuracy               2 29 27 1023 255 186 148


In [1]:
833+86+824+1541+1037+563+1034

5918

In [2]:
215+573+82+3041+846+625+536

5918

In [39]:
x = (a==b)

In [40]:
x

array([False, False,  True, ..., False, False,  True], dtype=bool)

In [98]:
a=1
a+=1
a

2

In [9]:
train_labels

array([ 2.,  3.,  6., ...,  2.,  3.,  5.], dtype=float32)

In [6]:
trainLablePD = pd.DataFrame()

In [7]:
trainLablePD['trainLable'] = train_labels

In [8]:
trainLablePD['trainLable'].value_counts

<bound method IndexOpsMixin.value_counts of 0        2.0
1        3.0
2        6.0
3        2.0
4        4.0
5        6.0
6        0.0
7        6.0
8        4.0
9        3.0
10       6.0
11       0.0
12       1.0
13       2.0
14       0.0
15       6.0
16       3.0
17       6.0
18       1.0
19       5.0
20       6.0
21       3.0
22       3.0
23       4.0
24       0.0
25       4.0
26       3.0
27       4.0
28       3.0
29       2.0
        ... 
10244    6.0
10245    6.0
10246    2.0
10247    3.0
10248    3.0
10249    6.0
10250    4.0
10251    2.0
10252    2.0
10253    6.0
10254    1.0
10255    6.0
10256    1.0
10257    1.0
10258    3.0
10259    2.0
10260    6.0
10261    0.0
10262    2.0
10263    6.0
10264    3.0
10265    0.0
10266    4.0
10267    4.0
10268    2.0
10269    1.0
10270    4.0
10271    2.0
10272    3.0
10273    5.0
Name: trainLable, dtype: float32>

In [27]:
a.reshape(-1,100)

array([[    0,     1,     2, ...,    97,    98,    99],
       [  100,   101,   102, ...,   197,   198,   199],
       [  200,   201,   202, ...,   297,   298,   299],
       ..., 
       [99700, 99701, 99702, ..., 99797, 99798, 99799],
       [99800, 99801, 99802, ..., 99897, 99898, 99899],
       [99900, 99901, 99902, ..., 99997, 99998, 99999]])

In [22]:
a


array([    0,     1,     2, ..., 99997, 99998, 99999])